In [72]:
import os
import re
import gc
import logging
import tempfile
import pdfplumber
import camelot
import PyPDF2
import pandas as pd
from tabulate import tabulate
from django.utils import timezone
from sqlite3 import Row
from pathlib import Path
from typing import Dict, Any, Optional, Type, List
from collections import defaultdict
# from ..tasks.parser import write_transactions


class CapitronBankParser():
    """
    Parser for the TDB regular statement format.
    """

    def __init__(
        self,
        sources_dir: str = "sources",
        results_dir: str = "results",
        # model_class: Optional[Type[Model]] = None,
        name_pattern: str = r'Дансны нэр\s*([^\n]+)',
        account_pattern: str = r'Дансны дугаар\s*MN(\d+)\s*',
        account_type_pattern: str = r'Эхний үлдэгдэл.*\b([A-Z]{3})\b',
        table_start_pattern: str = r'Огноо',
        table_end_pattern: str = r'Огноо Гүйлгээ хийсэн банк',
    ):
        # super().__init__(sources_dir, results_dir)
        self.name_pattern = name_pattern
        self.account_pattern = account_pattern
        self.account_type_pattern = account_type_pattern
        self.table_start_pattern = table_start_pattern
        self.table_end_pattern = table_end_pattern
        self.statement_parser = 'capitronbank'

    def write_transactions(self, df, settings, account):
        # Create a list of transactions to be bulk created
        transactions = []
        for _, row in df.iterrows():
            transaction_data = {
                'account': account,
            }
            for field, column_name in settings['columns'].items():
                try:
                    if field == 'date':
                        value = ''
                        if isinstance(column_name, dict):
                            for _, col in column_name.items():
                                value += " " + row[col]
                        else:
                            value = row[column_name]
                        if isinstance(value, str):
                            value = value.replace(" : ", " ")
                        naive_datetime = pd.to_datetime(value, errors='coerce')
                        # transaction_data[field] = timezone.make_aware(naive_datetime) if naive_datetime is not pd.NaT else None
                    elif field in ['income', 'expense', 'balance_end']:
                        value = row[column_name]
                        if pd.isna(value):
                            transaction_data[field] = None
                        elif value == "-":
                            transaction_data[field] = None
                        else:
                            if isinstance(value, str):
                                chars_to_replace = [',', '\n']
                                for char in chars_to_replace:
                                    value = value.replace(char, '')
                            value = 0 if value == "" else float(value)
                            if field == 'expense':
                                value = -abs(value)
                            transaction_data[field] = value
                    else:
                        transaction_data[field] = row[column_name]

                except Exception as e:
                    print("why------>", e)

            # Check if date is not null and at least one of income or expense has a value
            # if transaction_data.get('date') not in [None, ""] and (transaction_data.get('income') or transaction_data.get('expense')):
            transactions.append(transaction_data)

        # print(tabulate([t.__dict__ for t in transactions], headers="keys", tablefmt="grid"), flush=True)

        # Bulk create transactions
        # initial_rows_number = Transaction.objects.count()
        # Transaction.objects.bulk_create(transactions)

        # return Transaction.objects.count() - initial_rows_number
        return transactions

    def extract_customer_details(self, text: str) -> Dict[str, str]:
        """Extract customer details from the text content."""

        details = {
            'first_name': '',
            'last_name': '',
            'account_number': '',
            'account_type': '',
            'statement_parser': self.statement_parser
        }

        try:
            # Extract customer name
            name_match = re.search(self.name_pattern, text)
            print("name_match: ", name_match)
            if name_match:
                full_name = name_match.group(1).strip()
                # Split the name into first and last name
                name_parts = full_name.split()
                if len(name_parts) >= 2:
                    details['last_name'] = name_parts[0]
                    details['first_name'] = name_parts[1]

            # Extract account number
            account_match = re.search(self.account_pattern, text)
            if account_match:
                details['account_number'] = account_match.group(1)

            # Extract account type
            account_type_match = re.search(self.account_type_pattern, text)
            if account_type_match:
                details['account_type'] = account_type_match.group(0).strip()[-3:]

            print("details: ", details)

        except Exception as e:
            print(f"Error extracting customer details: {str(e)}")

        return details

    def extract_statement_details(self, pdf_file: str, settings: str, account: str, start_page: int = None, end_page: int = None) -> List[Dict[str, Any]]:
        """
        Extract statement details from the text content.

        Args:
            pdf_file: Either a file path string or a file object
            start_page (int, optional): Starting page number (1-indexed). If None, starts from page 1.
            end_page (int, optional): Ending page number (1-indexed). If None, processes all pages.

        Returns:
            List[Dict[str, Any]]: List of parsed transactions
        """

        try:
            # Determine page range for Camelot
            if start_page is not None and end_page is not None:
                # Convert to Camelot's page format (comma-separated string)
                page_range = ','.join(str(i) for i in range(start_page, end_page + 1))
                print(f"Capitron pdf processing pages: {page_range}")
            elif start_page is not None:
                page_range = f"{start_page}-end"
                print(f"Capitron pdf processing pages: {page_range}")
            elif end_page is not None:
                page_range = f"1-{end_page}"
                print(f"Capitron pdf processing pages: {page_range}")
            else:
                page_range = 'all'
                print("Capitron pdf processing all pages")

        except Exception as e:
            print(f"Error extracting tables from PDF: {str(e)}")
            return []

        print(f"Found {len(pdf_file.pages)} tables")

        converted_tran = []

        for page_num in range(start_page - 1, end_page):
            print("iterate num: ", page_num)
            table = pdf_file.pages[page_num]

            table = table.extract_tables({
                "vertical_strategy": "lines",  # Strategy for finding columns
                "horizontal_strategy": "lines",  # Strategy for finding rows
            })

            if table:
                for idx, row in enumerate(table):
                    if idx == 0:
                        df = pd.DataFrame(row[1:], columns=row[0])
                        headers = row[0]
                    else:
                        df = pd.DataFrame(row, columns=headers)

                    converted_tran.append(df)
                    print(f"Table {idx + 1} on Page {page_num + 1}:", flush=True)
                    # print(tabulate(df, headers="keys", tablefmt="grid"), flush=True)
            else:
                print(f"No tables found on Page {page_num + 1}.", flush=True)

        print(headers, flush=True)

        # Concatenate all DataFrames into a single DataFrame
        merged_df = pd.DataFrame()
        if converted_tran:
            merged_df = pd.concat(converted_tran, ignore_index=True)
            print("Merged DataFrame:", flush=True)
            # print(tabulate(merged_df, headers="keys", tablefmt="grid"), flush=True)
        else:
            print("No tables found in the entire PDF.", flush=True)

        print("Columns > > > > > >", merged_df.columns, flush=True)

        try:
            created_transactions = self.write_transactions(merged_df, settings, account)

        except Exception as e:
            print("whyyy", e)


        return created_transactions

    def parse_pdf(self, pdf_path: Path, register_number: str, request=None, start_page: int = None, end_page: int = None, chunk_size: int = 10) -> str:
        """Parse a single PDF file and extract its text content."""

        pdf_path = "D:\Projects\pdf-statement-parser\huulga_pdfs\capitron2025Aug\statement-2025-08-24 12_27_40.pdf"

        result = {
            'id': "",
            'name': "", # full name
            'account': "",
            'created_transactions': 0
        }

        try:
            with pdfplumber.open(pdf_path) as pdf:
                # Get first page for customer details
                first_page = pdf.pages[0]
                first_page_text = first_page.extract_text()

                # Extract and save customer details
                customer_details = self.extract_customer_details(first_page_text)
                # customer, account = self.save_customer_details(customer_details, register_number, request)
                account = customer_details['account_number']
                customer = customer_details


                if account:
                    print(f"Successfully saved customer details for account {account}")
                    # settings = BankStatementSettings.objects.get(bank=bank, type="pdf")

                    settings = {
                        "columns": {
                            "date": "Огноо",
                            "income": "Орлого",
                            "expense": "Зарлага",
                            "balance_end": "Үлдэгдэл",
                            "description": "Гүйлгээний утга",
                            "related_account": "Харьцсан данс / Нэр"
                        }
                    }

                    # Determine page range for processing
                    total_pages = len(pdf.pages)
                    start_idx = (start_page - 1) if start_page is not None else 0
                    end_idx = end_page if end_page is not None else total_pages

                    # Validate page range
                    start_idx = max(0, start_idx)  # Ensure start is not negative
                    end_idx = min(total_pages, end_idx)  # Ensure end doesn't exceed total pages

                    if start_idx >= end_idx:
                        print(f"Invalid page range: start_page={start_page}, end_page={end_page}, total_pages={total_pages}")
                        return result

                    print(f"Capitron pdf processing pages {start_idx + 1} to {end_idx} of {total_pages} total pages in chunks of {chunk_size}")

                    # Process pages in chunks to prevent memory issues
                    total_transactions = 0
                    chunk_start = start_idx

                    while chunk_start < end_idx:
                        chunk_end = min(chunk_start + chunk_size, end_idx)
                        print(f"Capitron pdf processing chunk: pages {chunk_start + 1} to {chunk_end}")

                        # Extract transactions from this chunk using Camelot
                        chunk_transactions = self.extract_statement_details(pdf, settings, account, chunk_start + 1, chunk_end)
                        print(f"Capitron pdf chunk {chunk_start + 1}-{chunk_end}: found {len(chunk_transactions)} transactions")

                        # Save transactions from this chunk
                        if chunk_transactions:
                            # if self.save_transactions(chunk_transactions, account):
                            total_transactions += len(chunk_transactions)
                            print(f"Capitron pdf chunk {chunk_start + 1}-{chunk_end}: successfully saved {len(chunk_transactions)} transactions")
                            # else:
                            #     print(f"Capitron pdf chunk {chunk_start + 1}-{chunk_end}: failed to save transactions")

                        # Clear chunk data from memory
                        del chunk_transactions
                        gc.collect()  # Force garbage collection

                        chunk_start = chunk_end

                    print(f"Capitron pdf successfully processed {total_transactions} total transactions from pages {start_idx + 1}-{end_idx}")
                else:
                    print(f"Failed to save customer details for {pdf_path.name}")

                # result['id'] = str(customer['id'])
                result['name'] = customer['last_name'] + " " + customer['first_name']
                result['account'] = account['account_number']
                result['created_transactions'] = total_transactions

        except Exception as e:
            print(f"Error parsing {pdf_path}: {str(e)}")

        return result


<>:222: SyntaxWarning: invalid escape sequence '\P'
<>:222: SyntaxWarning: invalid escape sequence '\P'
C:\Users\tnyam\AppData\Local\Temp\ipykernel_19464\3746501387.py:222: SyntaxWarning: invalid escape sequence '\P'
  pdf_path = "D:\Projects\pdf-statement-parser\huulga_pdfs\capitron2025Aug\statement-2025-08-24 12_27_40.pdf"


In [73]:
parser = CapitronBankParser()
parser.parse_pdf("D:\\Projects\\pdf-statement-parser\\huulga_pdfs\\05-khasbank.pdf", "УО02303134")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

name_match:  <re.Match object; span=(88, 136), match='Дансны нэр ЭНХСАРУУЛ ДАВААЦЭРЭН Огноо 2025-08-24'>
details:  {'first_name': 'ДАВААЦЭРЭН', 'last_name': 'ЭНХСАРУУЛ', 'account_number': '580030003019055588', 'account_type': 'MNT', 'statement_parser': 'capitronbank'}
Successfully saved customer details for account 580030003019055588
Capitron pdf processing pages 1 to 30 of 30 total pages in chunks of 10
Capitron pdf processing chunk: pages 1 to 10
Capitron pdf processing pages: 1,2,3,4,5,6,7,8,9,10
Found 30 tables
iterate num:  0
Table 1 on Page 1:
iterate num:  1
Table 1 on Page 2:
iterate num:  2
Table 1 on Page 3:
iterate num:  3
Table 1 on Page 4:
iterate num:  4
Table 1 on Page 5:
iterate num:  5
Table 1 on Page 6:
iterate num:  6
Table 1 on Page 7:
iterate num:  7
Table 1 on Page 8:
iterate num:  8
Table 1 on Page 9:
iterate num:  9
Table 1 on Page 10:
['Огноо', 'Зарлага', 'Орлого', 'Ханш', 'Харьцсан данс / Нэр', 'Үлдэгдэл', 'Гүйлгээний утга']
Merged DataFrame:
Columns > > > > >

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def


Error parsing D:\Projects\pdf-statement-parser\huulga_pdfs\capitron2025Aug\statement-2025-08-24 12_27_40.pdf: string indices must be integers, not 'str'


{'id': '',
 'name': 'ЭНХСАРУУЛ ДАВААЦЭРЭН',
 'account': '',
 'created_transactions': 0}